# 역전파 과정 코딩하기

신경망 모형에서는 역전파 과정을 통해 파라미터를 업데이트하여 성능을 높입니다.  
파이토치 프레임워크에서는 손실함수를 정의하고 `loss.backward()` 와 같은 메서드로 쉽게 역전파가 가능하지만, 이해가 부족했고 직접 해볼 필요성을 느껴서 파이토치의 텐서를 이용하여 역전파 과정을 짜보고자 합니다.

## 참고

[모두를 위한 딥러닝 시즌2](https://www.youtube.com/watch?v=B3VG-TeO9Lk&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=7)

[cs231n Stanford University](https://www.youtube.com/watch?v=i94OvYb6noo&t=989s)

### 과정
1. input layer에 feature의 수만큼 노드를 생성
2. N 개의 hidden layer에 직접 노드를 생성
3. 각 hidden layer를 통과할 때마다 활성화 함수를 거침
4. output layer에 얻고 싶은 결과에 따라 노드를 생성
5. chain rule에 의해 gradient 값을 구하고 gradient descent 방법으로 파라미터 업데이트
6. 위의 과정 반복

In [38]:
import torch

In [39]:
torch.__version__
device = "cpu"

In [57]:
# input layer 
X = torch.Tensor([[0,0], [0,1], [1,0], [1,1]]).to(device)
y = torch.Tensor([[0], [1], [1], [0]]).to(device)

# weight & bias
w1 = torch.Tensor([[1,1], [1,1]]).to(device)
b1 = torch.Tensor([1,2]).to(device)
w2 = torch.Tensor([[2],[1]]).to(device)
b2 = torch.Tensor([1]).to(device)

# Activation function
def sigmoid(x):
    return 1 / (1+torch.exp(-x))

# Activation function prime
def sigmoid_p(x):
    return sigmoid(x) * (1-sigmoid(x))

# Loss function (binary cross entropy)
def CE(y, y_pred):
    return torch.mean(y * torch.log(y_pred) + (1-y) * torch.log(1-y_pred))

In [58]:
b1.size()

torch.Size([2])

**Sigmoid**
$$ sigmoid(x) = \frac{1}{1+e^{-x}}$$  
$$ \frac{d}{dx}sigmoid(x) = \frac{1}{1+e^{-x}} \times \frac{e^{-x}}{1+e^{-x}}$$ 

**Binary Cross Entropy**
$$ Loss = -\frac{1}{N}\Sigma{y_i\cdot log(p(y_i)) + (1-y_i)\cdot log(1-p(y_i))} $$



In [59]:
EPOCH = 10000
lr = 0.01
for epoch in range(1,EPOCH+1):
    # 순전파 
    l1 = torch.add(torch.matmul(X,w1),b1) # w1x + b1
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1,w2),b2)
    y_pred = sigmoid(l2)
    
    loss = CE(y,y_pred)
    
    #역전파
    # layer 2
    d_loss = (y/y_pred) - (1-y)/(1-y_pred)
    d_l2 = sigmoid_p(l2) * d_loss
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_l2) # 0,1 -> 행과 열 변경
    
    # layer 1
    d_a1 = torch.matmul(d_l2, torch.transpose(w2, 0, 1))
    d_l1 = sigmoid_p(l1) * d_a1
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X, 0, 1), d_l1)
    
    # gradient descent
    w1 -= lr * d_w1
    b1 -= lr * torch.mean(d_b1,0)
    w2 -= lr * d_w2
    b2 -= lr * torch.mean(d_b2,0)
    
    # result print
    if epoch % 500 == 0:
        print(f"EPOCH : {epoch} / 10000 ({epoch/10000:.0f}%) \t Loss : {loss}")

EPOCH : 500 / 10000 (0%) 	 Loss : nan
EPOCH : 1000 / 10000 (0%) 	 Loss : nan
EPOCH : 1500 / 10000 (0%) 	 Loss : nan
EPOCH : 2000 / 10000 (0%) 	 Loss : nan
EPOCH : 2500 / 10000 (0%) 	 Loss : nan
EPOCH : 3000 / 10000 (0%) 	 Loss : nan
EPOCH : 3500 / 10000 (0%) 	 Loss : nan
EPOCH : 4000 / 10000 (0%) 	 Loss : nan
EPOCH : 4500 / 10000 (0%) 	 Loss : nan
EPOCH : 5000 / 10000 (0%) 	 Loss : nan
EPOCH : 5500 / 10000 (1%) 	 Loss : nan
EPOCH : 6000 / 10000 (1%) 	 Loss : nan
EPOCH : 6500 / 10000 (1%) 	 Loss : nan
EPOCH : 7000 / 10000 (1%) 	 Loss : nan
EPOCH : 7500 / 10000 (1%) 	 Loss : nan
EPOCH : 8000 / 10000 (1%) 	 Loss : nan
EPOCH : 8500 / 10000 (1%) 	 Loss : nan
EPOCH : 9000 / 10000 (1%) 	 Loss : nan
EPOCH : 9500 / 10000 (1%) 	 Loss : nan
EPOCH : 10000 / 10000 (1%) 	 Loss : nan


역전파시에 행렬 연산이 이해가 안됨